In [1]:
HTML(read(open("style.css"), String))

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\";\r\n    font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

# Alpha-Beta Pruning


This notebook implements an AI which calculates the best move for a chess game using alpha-beta-Pruning algorithm.

In [2]:
using Pkg
# Pkg.add("Chess")
using Chess
using Random

# Pkg.add("NBInclude")
using NBInclude

In [3]:
@nbinclude("EvaluatePosition.ipynb")

terminal_evaluation (generic function with 1 method)

## AlphaBetaMax

The `alphaBetaMax_noMem` function takes in 3 arguments and 2 optional arguments.
1. `State` is a chess `state` of type `Board`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating 
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process

The function returns the maximal centipawn evaluation of the current position for the player playing white where both players have played the optimal moves according to the algorithm and terminating after the given depth. This function does not use Memoization.

In [4]:
function alphaBetaMax_noMem(State, score, depth, alpha = -Inf, beta = Inf)
    if isterminal(State)
        return terminal_evaluation(State) - depth
    end
    if depth == 0
        return score
    end
    for move in moves(State)
        nextEval = evaluate_move(State, move, score)
        undoinfo = domove!(State, move)
        value = alphaBetaMin_noMem(State, nextEval, depth - 1, alpha, beta)
        undomove!(State, undoinfo)
        if value >= beta
            return value
        end
        alpha = max(alpha, value)
    end
    return alpha
end

alphaBetaMax_noMem (generic function with 3 methods)

## AlphaBetaMin

The `alphaBetaMin_noMem` function takes in 3 arguments and 2 optional arguments.
1. `State` is a chess `state` of type `Board`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating 
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process

The function returns the minimal centipawn evaluation of the current position for the player playing black where both players have played the optimal moves according to the algorithm and terminating after the given depth. This function does not use Memoization.

In [5]:
function alphaBetaMin_noMem(State, score, depth, alpha = -Inf, beta = Inf)
    if isterminal(State)
        return terminal_evaluation(State) + depth
    end
    if depth == 0
        return score
    end
    for move in moves(State)
        nextEval = evaluate_move(State, move, score)
        undoinfo = domove!(State, move)
        value = alphaBetaMax_noMem(State, nextEval, depth - 1, alpha, beta)
        undomove!(State, undoinfo)
        if value <= alpha
            return value
        end
        beta = min(beta, value)
    end
    return beta
end

alphaBetaMin_noMem (generic function with 3 methods)

## Alpha-beta Pruning function

The `alphaBetaPruning_noMem` function takes in 3 arguments
1. `State` is the current state of type `Board`
1. `score` is the static centipawn evaluation of the static position
1. `depth` is the number of halfmoves the engine should analyze before terminating

The function returns the best value and the best move the moving player can play in the current position. It calls the alpha-beta-pruning algorithm. If multiple moves are found which result in the best evaluation a random move will be chosen. This function does not use Memoization.

In [20]:
function alphaBetaPruning_noMem(State, score, depth)
    next_moves = moves(State)
    BestMoves = []
    if sidetomove(State) == WHITE
        bestVal = alphaBetaMax_noMem(State, score, depth)
        for move in next_moves 
            nextEval = evaluate_move(State, move, score)
            undoinfo = domove!(State, move) 
            if alphaBetaMin_noMem(State, nextEval, depth - 1) == bestVal
                append!(BestMoves, [move])
            end
            undomove!(State, undoinfo)
        end
    elseif sidetomove(State) == BLACK
        bestVal = alphaBetaMin_noMem(State, score, depth)
        for move in next_moves 
            nextEval = evaluate_move(State, move, score)
            undoinfo = domove!(State, move)
            if alphaBetaMax_noMem(State, nextEval, depth - 1) == bestVal
                append!(BestMoves, [move])
            end
            undomove!(State, undoinfo)
        end
    end
    BestMove = rand(BestMoves)
    return bestVal, BestMove
end

alphaBetaPruning_noMem (generic function with 1 method)

## Alpha-beta-Pruning with Memoization

In [22]:
@nbinclude("ZobristHashing.ipynb")

zobrist_hash (generic function with 1 method)

In [23]:
zobristHasher = generate_zobrist_hashing()

ZobristHashing(UInt64[0x12dcb60454d2ac0d 0x32afcbc4f40f8326 … 0x1f3e93b738de1471 0x16a28bd1aa580032; 0x16e13c0569940566 0x26e4447cf43da01d … 0x5e3914ada364d3e4 0x477eb6a6768436ee; … ; 0x446134032571cdc5 0x5c9e7ec986da4fa9 … 0x52359501724eebbd 0x39886bddccf99ada; 0xec2bfe25dd7663fa 0x06d2b1ce2a01c10f … 0x8f5293cac7d6f25b 0xce6de562282eea78], UInt64[0x52c6605035f5c697, 0x5d8efd62ab05d30b, 0x3fc61292ceb8b5c3, 0x03ec26307105d654], UInt64[0x05f82d3373db6a50, 0x63b06d75e5678abd, 0xf19a954d3cd6c6da, 0xc6b505d6c62a8bb2, 0x62357b75b4b976fe, 0x358870e3587edba2, 0xc07d1971a8a6ae56, 0x9799c1e03d5ef7e4], 0xfad2e4f6ea9da8aa)

### alphaBetaMax function

The `alphaBetaMax` function takes in 3 arguments and 2 optional arguments.
1. `State` is a chess `state` of type `Board`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating 
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process

The function returns the maximal centipawn evaluation of the current position for the player playing white where both players have played the optimal moves according to the algorithm and terminating after the given depth. This function does use Memoization meaning it saves and uses calculated values stored the `gCache`.

In [24]:
function alphaBetaMax(State, score, depth, alpha = -Inf, beta = Inf)
    if isterminal(State)
        return terminal_evaluation(State) - depth
    end
    if depth == 0
        return score
    end
    for move in moves(State)
        nextEval = evaluate_move(State, move, score)
        undoinfo = domove!(State, move)
        value = evaluate(State, alphaBetaMin, nextEval, depth - 1, alpha, beta)
        undomove!(State, undoinfo)
        if value >= beta
            return value
        end
        alpha = max(alpha, value)
    end
    return alpha
end

alphaBetaMax (generic function with 3 methods)

### alphaBetaMin function

The Alpha-Beta-Min function takes in 3 arguments and 2 optional arguments.
1. `State` is a chess `state` of type `Board`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating 
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process

The function returns the minimal centipawn evaluation of the current position for the player playing black where both players have played the optimal moves according to the algorithm and terminating after the given depth. This function does use Memoization meaning it saves and uses calculated values stored the `gCache`.

In [25]:
function alphaBetaMin(State, score, depth, alpha = -Inf, beta = Inf)
    if isterminal(State)
        return terminal_evaluation(State) + depth
    end
    if depth == 0
        return score
    end
    for move in moves(State)
        nextEval = evaluate_move(State, move, score)
        undoinfo = domove!(State, move)
        value = evaluate(State, alphaBetaMax, nextEval, depth - 1, alpha, beta)
        undomove!(State, undoinfo)
        if value <= alpha
            return value
        end
        beta = min(beta, value)
    end
    return beta
end

alphaBetaMin (generic function with 3 methods)

### evaluate function

Initialize global Cache `gCache`

In [26]:
gCache = Dict()

Dict{Any, Any}()

The `evaluate` function adds memoization to the `alphaBetaMax` and `alphaBetaMin` function.

It takes in 4 arguments and 2 optional arguments.
1. `State` is the current state represented by a `Board`
1. `f` takes in either the function `alphaBetaMax` and `alphaBetaMin`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process

The function `evaluate` evaluates the same result as the function `f`. Additionally it saves calculated results in the `gCache`. And uses any entries in the `gCache` if the same function has already been called.

In [6]:
function evaluate(State, f, score, depth, alpha = -Inf, beta = Inf)
    global gCache
    hash = zobrist_hash(State, zobristHasher)
    if hash in keys(gCache) 
        flag, v, d = gCache[hash]
        if d >= depth   # check if stored depth to this position is already higher than required
            if flag == "="
                return v
            elseif flag == "<="
                if v <= alpha
                    return v
                else
                    beta = min(beta, v)
                    w = f(State, score, depth, alpha, beta)
                    store_cache(hash, depth, alpha, beta, w)
                    return w
                end
            elseif flag == ">="
                if beta <= v
                    return v
                else
                    alpha = max(alpha, v)
                    w = f(State, score, depth, v, beta)
                    store_cache(hash, depth, v, beta, w)
                    return w
                end
            end
        end
    end
    # no value stored in gCache for State or depth of stored State has less depth than required
    v = f(State, score, depth, alpha, beta)
    store_cache(hash, depth, alpha, beta, v)
    return v
end

evaluate (generic function with 4 methods)

The `store_cache` is a helping function that stores values into the `gCache`. 

It takes 5 arguments:
1. `State` is the current `state` of type `Board`
1. `depth` is the number of halfmoves the engine should analyze before terminating
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process
1. `v` is the value that got calculated by the `evaluate` function that needs to be stored


In [13]:
function store_cache(hash, depth, alpha, beta, v)
    global gCache
    if v <= alpha
        gCache[hash] = ("<=", v, depth)
    elseif v < beta
        gCache[hash] = ("=", v, depth)
    else # beta <= v
        gCache[hash] = (">=", v, depth)
    end
end

store_cache (generic function with 1 method)

The `alphaBetaPruning` function takes in 3 arguments
1. `State` is the current state of type `Board`
1. `score` is the static centipawn evaluation of the static position
1. `depth` is the number of halfmoves the engine should analyze before terminating

The function returns the best value and the best move the moving player can play in the current position. It calls the alpha-beta-pruning algorithm. If multiple moves are found which result in the best evaluation a random move will be chosen.

In [30]:
function alphaBetaPruning(State, score, depth)
    next_moves = moves(State)
    BestMoves = []
    if sidetomove(State) == WHITE
        bestVal = evaluate(State, alphaBetaMax, score, depth)
        for move in next_moves
            nextEval = evaluate_move(State, move, score)
            undoinfo = domove!(State, move)
            if evaluate(State, alphaBetaMin, nextEval, depth - 1) == bestVal
                append!(BestMoves, [move])
            end
            undomove!(State, undoinfo)
        end
    elseif sidetomove(State) == BLACK
        bestVal = evaluate(State, alphaBetaMin, score, depth)
        for move in next_moves 
            nextEval = evaluate_move(State, move, score)
            undoinfo = domove!(State, move)
            if evaluate(State, alphaBetaMax, nextEval, depth - 1) == bestVal
                append!(BestMoves, [move])
            end
            undomove!(State, undoinfo)
        end
    end
    BestMove = rand(BestMoves)
    return bestVal, BestMove
end

alphaBetaPruning (generic function with 1 method)